

# 🛠️ **Interactive OpenAI Function Calling Tutorial** 

Learn by doing! This interactive tutorial walks you through creating a function, and then lets you chat with an assistant to use it.

Get an intuitive understanding of Function Calling in under 5 minutes!

---

## **What is Function Calling?**

Function calling is a powerful feature that enables a ChatGPT assistant to *act*. By defining specific abilities (or functions), you grant the assistant the capability to perform them based on a user's request.

---

## **What to Expect in this Notebook**

In this notebook, we'll look at examples to understand how function calling works:

1. 🎈 **Simple Example:** Balloon launch function
2. 🌦️ **Advanced Example:** Fetch weather info function

---

### **About**
👤 **Creator:** [Josh Bickett](https://twitter.com/josh_bickett)

📘 **Colab:** [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/joshbickett/function-calling-notebook/blob/main/interactive-function-calling-notebook.ipynb)

🔗 **Github:** [View on GitHub](https://github.com/joshbickett/function-calling-notebook)



## Installing the OpenAI Library

Run the code below to install the `openai` library, which lets us interact with OpenAI's platform.


In [ ]:
!pip install openai

## Setting Up the Environment

In this section, we will import the necessary libraries to make our function work smoothly. Among them, you'll notice the `'YOUR_API_KEY'` placeholder. 

To use the OpenAI platform, you need to replace `'YOUR_API_KEY'` with your actual OpenAI API key. If you don't have one, you can obtain an OpenAI key [here](https://platform.openai.com/account/api-keys).


In [18]:

import requests
import openai
import json
import random

openai.api_key = 'YOUR_API_KEY'

## Example 1: Launching a Water Balloon 🎈💦

This is a playful function we've built for demonstration. It simulates launching a water balloon of a given size and then randomly decides where the balloon hits. Will it be the ground, a tree, or even the moon? Let's find out!


In [15]:

def launch_water_balloon(balloon_size):
    hit_locations = ['the ground', 'a tree', 'the moon']
    random_location = random.choice(hit_locations)
    launch_results = f'Launching the {balloon_size} water balloon, it hit {random_location}!'
    return launch_results


### Saving the Function for ChatGPT 🤖

After defining our function, we need to create a `functions` list variable. This list will be passed to OpenAI's `ChatCompletion.create` method, letting the assistant know what "abilities" it has.

In [6]:
functions = [
    {
        "name": "launch_water_balloon",
        "description": "This function launches a hypothetical water balloon",
        "parameters": {
            "type": "object",
            "properties": {
                "balloon_size": {
                    "type": "string",
                    "enum": ["Large", "Medium", "Small"],
                },
            },
            "required": ["balloon_size"],
        },
    }
]

available_functions = {
    "launch_water_balloon": launch_water_balloon,
}

### Create the "Chat Environment" 🤖➡️🛠️

Below, we define `converse` to send the user's message to the OpenAI API, including information about our previously created water balloon launcher function.

The code `response_message.get("function_call")` checks if the `response_message` is a function calling response. If it is, then the assistant is requesting to use a function, such as the balloon launcher, instead of replying to the user with a message.


In [20]:

def converse(messages):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]
    
    
    messages.append(response_message) 
    if response_message.get("function_call"):
        function_name = response_message["function_call"]["name"]
        function_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])

        print('[inspector] Oh, we got a function call! 🔨')
        print('[inspector] ===> function_name', function_name)
        print('[inspector] ===> function_args', function_args)
        function_response = function_to_call(
            balloon_size=function_args.get("balloon_size"),
        )

        print('[inspector] ===> function_response', function_response)


        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )

        print('Assistant: ', function_response)
        
    else: 
        content = response_message.get("content")
        print('Assistant: ', content)

    

### Now we're ready, let's chat with ChatGPT! 🎉

Now that everything's set up, it's time to start a conversation.

Simply run the code cell and type your message, and the Assistant will respond, potentially even launching a water balloon or two! 😄


In [ ]:
messages = []
message_count = 0 
while message_count < 10: 
    user_input = input("User: ")
    user_message = {"role": "user", "content": user_input}
    messages.append(user_message)
    converse(messages)
    message_count += 1

## Example 2: Fetching Tonight's Weather 🌙☁️

This function is a step up from our playful balloon launch, diving into the practical use case of fetching weather data for major US cities.

We have a predefined list of cities with their coordinates. When given a city name:
1. The function first locates the city within our list.
2. Using the city's coordinates, it queries the National Weather Service's API for the weather forecast.
3. It then extracts and returns the detailed forecast for tonight from the data received.

Want to know if it's a good night for stargazing in San Francisco or if you should carry an umbrella in New York City? Just ask, and let ChatGPT fetch the weather details for you!


##### ***Important note***:
You have to stop the last cell in order to continue because it continues running since it is a conversation



In [ ]:



cities = [
    {"name": "San Francisco", "latitude": 37.7749, "longitude": -122.4194},
    {"name": "New York City", "latitude": 40.7128, "longitude": -74.0060},
    {"name": "Los Angeles", "latitude": 34.0522, "longitude": -118.2437},
    {"name": "Chicago", "latitude": 41.8781, "longitude": -87.6298},
    {"name": "Miami", "latitude": 25.7617, "longitude": -80.1918}
]

def get_weather_data(city_name):
    # Find the city object by name
    city = next((c for c in cities if c["name"] == city_name), None)

    # If city is found, get the weather data using its coordinates
    if city:
        # Construct the URL based on the city's latitude and longitude
        url = f"https://api.weather.gov/points/{city['latitude']},{city['longitude']}"

        # Make a request to the URL
        response = requests.get(url)
        

        # Ensure the response status is 200 (OK)
        if response.status_code == 200:
            data = response.json()

            # Extracting the forecast URL from the response
            forecast_url = data["properties"]["forecast"]

            # Fetch the actual forecast data using the extracted URL
            forecast_response = requests.get(forecast_url)

            if forecast_response.status_code == 200:
                forecast_data = forecast_response.json()
                tonight_forecast = forecast_data['properties']['periods'][0]

                # Display the forecast
                return tonight_forecast['detailedForecast']
            else:
                print(f"Error fetching forecast data. HTTP Status Code: {forecast_response.status_code}")
                return None
        else:
            print(f"Error fetching grid point data. HTTP Status Code: {response.status_code}")
            return None
    else:
        print(f"City '{city_name}' not found in the list.")
        return None


### Updating Function Definitions 🔄

We're setting up the `get_weather_data` function. Similar to what we did with the balloon launcher, this allows ChatGPT to fetch weather details for select cities based on the given parameters.


In [ ]:
functions = [{
    "name": "get_weather_data",
    "description": "This function can get weather information for some of the US's largest cities",
    "parameters": {
        "type": "object",
        "properties": {
            "city_name": {
                "type": "string",
                "enum": ["San Francisco", "New York City", "Los Angeles", "Chicago", "Miami"],
            },
        },
        "required": ["city_name"],
    },
}]
available_functions = {
    "get_weather_data": get_weather_data,
}

### Refreshing the `converse` Function 🔄

We've tweaked our main conversation handler, `converse`, to account for the new weather-fetching capability. It now understands how to work with the `get_weather_data` function.


In [ ]:

def converse(messages):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]
    
    
    messages.append(response_message) 
    if response_message.get("function_call"):
        function_name = response_message["function_call"]["name"]
        function_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        print('[inspector] Oh, we got a function call! 🔨')
        print('[inspector] ===> function_name', function_name)
        print('[inspector] ===> function_args', function_args)
        function_response = function_to_call(
            city_name=function_args.get("city_name"),
        )
        print('[inspector] ===> function_response', function_response)


        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )

        print('Assistant: ', function_response)
        
    else: 
        content = response_message.get("content")
        print('Assistant: ', content)

    

      

### Chatting with ChatGPT: Let's Talk Weather! ☔🌞

Initiate another conversation with ChatGPT! This time, you can ask about real-time weather data for some of the major US cities. Experience the dynamic function calling in action!


In [ ]:
messages = []
message_count = 0 
while message_count < 10: 
    user_input = input("User: ")
    user_message = {"role": "user", "content": user_input}
    messages.append(user_message)
    converse(messages)
    message_count += 1

## Conclusion 🎉

Congratulations on completing this interactive tutorial on OpenAI's function calling with ChatGPT! You've successfully learned how to integrate dynamic function calls into your conversations with ChatGPT, empowering it to "act" based on your code and provide more dynamic responses.

If you found this Colab notebook useful and would like to see more content like this, consider following me on Twitter. 

🐦 [Follow Josh Bickett on Twitter](https://twitter.com/josh_bickett)

I also share software tutorials and LLM content on YouTube. 

🎥 [Subscribe to Josh Bickett on YouTube](https://www.youtube.com/@joshbickett)
